<table width="100%">
    <tr style="border-bottom:solid 2pt #009EE3">
        <td style="text-align:left" width="10%">
            <a href="emg_fatigue_evaluation_median_freq.dwipynb" download><img src="../../images/icons/download.png"></a>
        </td>
        <td style="text-align:left" width="10%">
            <a><img class="not_active_img" src="../../images/icons/program.png" title="Will be released soon !"></a>
        </td>
        <td></td>
        <td style="text-align:left" width="5%">
            <a href="../MainFiles/opensignalsfactory.ipynb"><img src="../../images/icons/home.png"></a>
        </td>
        <td style="text-align:left" width="5%">
            <a href="../MainFiles/contacts.ipynb"><img src="../../images/icons/contacts.png"></a>
        </td>
        <td style="text-align:left" width="5%">
            <a href="https://github.com/opensignalsfactory/opensignalsfactory"><img src="../../images/icons/github.png"></a>
        </td>
        <td style="border-left:solid 2pt #009EE3" width="20%">
            <img src="../../images/ost_logo.png">
        </td>
    </tr>
</table>

<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_4"><div id="image_img"
        class="header_image_4"></div></td>
        <!-- Available classes for "image_td" element:
        - header_image_color_1 (For Notebooks of "Open" Area);
        - header_image_color_2 (For Notebooks of "Acquire" Area);
        - header_image_color_3 (For Notebooks of "Visualise" Area);
        - header_image_color_4 (For Notebooks of "Process" Area);
        - header_image_color_5 (For Notebooks of "Detect" Area);
        - header_image_color_6 (For Notebooks of "Extract" Area);
        - header_image_color_7 (For Notebooks of "Decide" Area);
        - header_image_color_8 (For Notebooks of "Explain" Area);

        Available classes for "image_img" element:
        - header_image_1 (For Notebooks of "Open" Area);
        - header_image_2 (For Notebooks of "Acquire" Area);
        - header_image_3 (For Notebooks of "Visualise" Area);
        - header_image_4 (For Notebooks of "Process" Area);
        - header_image_5 (For Notebooks of "Detect" Area);
        - header_image_6 (For Notebooks of "Extract" Area);
        - header_image_7 (For Notebooks of "Decide" Area);
        - header_image_8 (For Notebooks of "Explain" Area);-->
        <td class="header_text">Fatigue Evaluation - Evolution of Median Power Frequency</td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">pre-process&#9729;emg</td>
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>


In contrast to the cardiac muscle, skeletal muscles are very susceptible to fatigue when exposed to an intense activity, for example, during a sports practice.

Fatigue <a href="https://en.wikipedia.org/wiki/Muscle_fatigue"><img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> can be defined as a complex physiological phenomenon with several causes and dependent of different mechanisms (accordingly to Cifrek et. al. <a href="https://www.clinbiomech.com/article/S0268-0033(09)00025-4/fulltext"><img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>).

Fatigue  may have serious consequences, being one triggering phenomenon behind muscular injuries.

So, taking into account the relevance of this phenomenon, fatigue study and monitoring is a promising research area, with some decades.
During this period, it was demonstrated that some parameters extracted from EMG signal evolve in a particular way as fatigue is being acquired.

One of this parameters, and probably the most consensual one, is the median power frequency <a href="http://www.soe.uoguelph.ca/webfiles/mleuniss/Biomechanics/EMG.html"><img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>, that decreases along fatigue acquisition, i.e. the power spectrum suffers a compression to lowest frequency components <a href="https://en.wikipedia.org/wiki/Muscle_fatigue#Electromyography"><img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>.

In this **<span class="color5">Jupyter Notebook</span>** it will be presented the basic methodology to monitoring the fatigue along time.


<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [1]:
# OpenSignals Tools own package for loading and plotting the acquired data
import opensignalsfactory as osf
import opensignalsfactory.signal_samples as signal_samples

# Scientific packages
from numpy import linspace, where
from scipy.signal import periodogram
from  scipy.integrate import cumtrapz

In [2]:
# Base packages used in OpenSignals Tools Notebooks for ploting data
from bokeh.plotting import output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import BoxAnnotation
output_notebook(hide_banner=True)

<p class="steps">2 - Load of acquired EMG data, collected during a fatigue induction trial (*biceps brachii* isometric contraction) </p>

In [3]:
# Load of data
data, header = signal_samples.load_signal("emg_fatigue")

<p class="steps">3 - Mac address identification for the device and the channel used during acquisition</p>

In [4]:
mac_address = list(header.keys())[0]
channel = "CH" + str(header[mac_address]["channels"][0])

print ("Mac Address: " + str(mac_address) + " Channel: " + str(channel))

Mac Address: 00:07:80:4C:01:B1 Channel: CH2


<p class="steps">4 - Sampling rate and acquired data samples storage (inside variables)</p>

In [5]:
# Sampling rate and acquired data
sr = header[mac_address]["sampling rate"]
device = header[mac_address]["device"]
resolution = header[mac_address]["resolution"][0]

# Signal Samples
signal = data[mac_address][channel]
time = linspace(0, len(signal) / sr, len(signal))

<p class="steps">5 - Muscular activations detection</p>
*Each muscular activation defines a processing window*

In [6]:
# The default call of detect_emg_activations function is:
# detect_emg_activations(emg_signal, sample_rate, smooth_level=20, threshold_level=10, time_units=False, volts=False,
# resolution=None, device="biosignalsplux", plot_result=False)
# This function returns the samples where each activation period starts and ends (first two outputs, reason why it is specified
# [:2]) the smoothed EMG signal samples and threshold level. Samples above the threshold are converted to 1 and the samples
# below to 0, which give rise to a rectangular activation signal.
activation_begin, activation_end = osf.detect_emg_activations(signal, sr)[:2]

osf.detect_emg_activations previous call has explicit and implicit arguments. The list containing the signal samples collected during the acquisition and the sampling rate at which the acquisition was carried out are the explicit ones, while *smoothLevel* and *thresholdLevel* correspond to implicit arguments that assume predefined values of 20 % and 10 %, respectively.

Changing these two parameters will produce changes in the detection of muscular activations, as can be seen at the following figure.

In [7]:
osf.plot_compare_act_config(signal, sr)

<p class="steps">6 - Extraction of the Median Power Frequency that characterises each muscular activation (processing window)</p>
*The Median Power Frequency is defined as the frequency value that allows the power spectrum division into two regions with equal power*

\begin{equation}
    \int_0^{f_{median}} PSD(f) df = \frac{1}{2} \int_0^{sr/2} PSD(f) df
\end{equation}
*where $f_{median}$ defines the median power frequency, $PSD(f)$ is the power spectral density estimate, after decomposing the signal by applying the Fourier Transform, for the elementary component with frequency f. The term $sr$ refers to the "sampling rate" abbreviation.*

In [8]:
# Iteration along muscular activations
median_freq_data = []
median_freq_time = []
for activation in range(0, len(activation_begin)):
    processing_window = signal[activation_begin[activation]:activation_end[activation]]
    central_point = (activation_begin[activation] + activation_end[activation]) / 2
    median_freq_time += [central_point / sr]

    # Processing window power spectrum (PSD) generation
    freqs, power = periodogram(processing_window, fs=sr)

    # Median power frequency determination
    area_freq = cumtrapz(power, freqs, initial=0)
    total_power = area_freq[-1]
    median_freq_data += [freqs[where(area_freq >= total_power / 2)[0][0]]]
    # The previous indexation [0][0] was specified in order to only the first sample that verifies
    # the condition area_freq >= total_power / 2 be returned (all the subsequent samples will verify
    # this condition, but, we only want the frequency that is nearest to the ideal frequency value
    # that divides power spectrum into to regions with the same power - which is not achievable in
    # a digital processing perspective).

<p class="steps">7 - Graphical Representation of the Median Frequency evolution time series</p>

In [9]:
osf.plot_median_freq_evol(time, signal, median_freq_time, median_freq_data, activation_begin, activation_end, sr)

*This procedure can be automatically done by **fatigue_eval_med_freq** function in **extract** module of **<span class="color2">opensignalsfactory</span>** package*

In [10]:
osf.fatigue_eval_med_freq(signal, sr)

Time (s)  Median Frequency (Hz)
0     2.7920               78.12500
1     7.0210               70.31250
2    11.2805               78.12500
3    15.1895               70.31250
4    19.3120               74.21875
5    23.1825               78.12500
6    27.1540               74.21875
7    31.3080               74.21875
8    35.2350               74.21875
9    39.1450               82.03125
10   42.8330               66.40625
11   47.0160               70.31250
12   50.9740               66.40625
13   54.9625               66.40625
14   59.1560               70.31250
15   62.9980               70.31250
16   67.3105               62.50000
17   71.2715               70.31250
18   75.2385               66.40625
19   79.1415               66.40625
20   82.9460               62.50000
21   86.8410               66.40625
22   90.8875               62.50000
23   95.0355               62.50000
24   98.9180               66.40625
25  103.0290               58.59375
26  107.2275               62.50000
27  110.9635               62.50000
28  115.1860               54.68750
29  119.4930               54.68750

<hr>
<table width="100%">
    <tr>
        <td style="border-right:solid 3px #009EE3" width="30%">
            <img src="../../images/ost_logo.png">
        </td>
        <td width="35%" style="text-align:left">
            <a href="https://github.com/opensignalsfactory/opensignalsfactory">&#9740; GitHub Repository</a>
            <br>
            <a href="../MainFiles/opensignalsfactory.ipynb">&#9740; Notebook Categories</a>
            <br>
            <a href="https://pypi.org/project/opensignalsfactory/">&#9740; How to install opensignalsfactory Python package ?</a>
            <br>
            <a href="../MainFiles/signal_samples.ipynb">&#9740; Signal Library</a>
        </td>
        <td width="35%" style="text-align:left">
            <a href="../MainFiles/by_diff.ipynb">&#9740; Notebooks by Difficulty</a>
            <br>
            <a href="../MainFiles/by_signal_type.ipynb">&#9740; Notebooks by Signal Type</a>
            <br>
            <a href="../MainFiles/by_tag.ipynb">&#9740; Notebooks by Tag</a>
            <br>
            <br>
        </td>
    </tr>
</table>

In [11]:
from opensignalsfactory.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................
